# CAPSTONE PROJECT- Battle of the Neighborhoods

### Applied Data Science Capstone by IBM/Coursera

Table of Contents:
* [Introduction](#introduction)
* [Data](#data)
* [Process](#process)
* [Foursquare Process](#4sq)
* [Results](#results)
* [Conclusion](#conclusion)

# Introduction <a name="introduction">
My Battle of the Neighborhood will take place in Washington, DC. My primary goal is to identify a prime location for a co-working space using a list of 13 locations around the city. The 13 locations are addresses that I self-identified as ideal given their underuse for their original intended purpose (primary school). 

To select wihc of the 13 locations is <i>ideal</i>, I am using FourSquare data to organize neighborhood ammenties, data from WMATA (DC Area Mass Transit), DC Government KML files, and a location file with 13 distinct locations in DC (Named using the Greek Alphabet Alpha to Nu). 


In the end, I am going to rank order the locations based on access to:
<ol type="1">
    <li>Mass transit</li>
    <li>Recreation</li>
    <li>Resturants</li>
    <li>Services</li>
    <li>Medical</li>
    <li>Education</li>
</ol>     
        
Given life in DC, I am weighting mass transit (from this point forward <i>"Metro"</i>) as the most valuable, followed by resturants, than education. The remaining categories will recieve no weighting. 

So this isn't just a battle of the neighborhoods, but a battle for for 13 unique locations in DC (representing many different geographic groupings). LET THE BATTLE BEGIN!

# Data <a name="data"></a>

As discussed above, selecting a location or neighborhood can never come down to a single point. In this case, I am using 6 defined categories and within those 6 categories, I am giving <i>weight</i> to
<ol type="1">
    <li>Mass transit</li>
    <li>Resturants</li>
    <li>Education</li>
</ol>

# External Data Sources
In order to complete this project I am using the following data sources:
<table style="width:100%">
  <tr>
    <th>DataFrame</th>
    <th>File Name</th>
    <th>Data Description</th>
    <th>Data Type</th>
    <th>Source</th>
  </tr>
  <tr>
    <td>Venues</td>
    <td>results</td>
    <td>Foursquare Data</td>
    <td>JSON</td>
    <td>Foursquare</td>
  </tr>
  <tr>
    <td>wmata_data</td>
    <td>json_data</td>
    <td>WMATA Data(better location data for Metro)</td>
    <td>JSON</td>
    <td>WMATA</td>
  </tr>
  <tr>
    <td>fp</td>
    <td>address_geo.csv</td>    
    <td>13 unique locations</td>
    <td>csv</td>
    <td>Personal</td>
  </tr>
  <tr>
    <td>dc_n</td>
    <td>Neighborhood_labels.csv</td>    
    <td>DC Neighborhood Geometry</td>
    <td>KML</td>
    <td>OpenData Washington, DC</td>
  </tr> 
</table>

# Process <a name="process">

My initial process used just Foursquare data, so the process was rather simple:
<ol type="1">
    <li>Google the center of Washington, DC and create lat/long data variables for DC</li>
    <li>Use those lat/long to limit Foursqure Data</li>
    <li>Process Foursquare Data by neighborhood in DC</li>
    <li>Use GIS python functions to identify which neighborhood my unique locations are located</li>
    <li>Total up the # of ammenities by neighborhood, per Foursquare</li>
    <li>Rank order locations</li>
    <li>Select the winner</li>
</ol>
    
Unfortunately I ran into four significant issues:
* [Too many business types in the Foursquare Data](#1)
* [DC has unique and complicated Geography](#2)
* [Foursquare Commuter data was not useful](#3)
* [Difficulties using GIS Pyton code](#4)

Therefore, my actual process was:
<ol type="1">
    <li>Google the center of Washington, DC and create lat/long data variables for DC</li>
    <li>Use those lat/long to limit Foursqure Data</li>
    <li>Process Foursquare Data by neighborhood in DC</li>
    <li>Group Foursquare data into 6 categories </li>
    <li>Use QGIS to identify which neighborhood my unique locations are located</li>
    <li>Load and process WMATA JSON data</li>
    <li>Use QGIS to merge different DC geometries (neighborhoods and cluster </li>
    <li>Total up the # of ammenities by neighborhood, per Foursquare and WMATA</li>
    <li>Weigh the totals for WMATA, Resturants, and Education to give those categories more value </li>
    <li>Rank order locations</li>
    <li>Select the winner</li>
</ol>

# First Steps-- loading libraries, establishing root directory, loading base files!

In [1]:
#load necessary libraries
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import requests
import matplotlib
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize
import os
import sys
print('Donzos')

Donzos


In [4]:
#create a var to store root_dir to load or write files
Root_dir = os.getcwd()

print(Root_dir)

/Users/jeremymcdonald/Desktop/OSP/Untitled Folder


In [5]:
#load addresses for identified locations
fp= 'address_geo.csv'
data=pd.read_csv(fp)


dc_n='Neighborhood_Labels.csv'
dc_data=pd.read_csv(dc_n)
print ('locked and loaded')

locked and loaded


# Foursquare Process <a name="4sq"></a>

The next section downloads and incorporates Foursquare Data for Washington, DC

As described above, I am doing the following:
<ol type="1">
    <li>Google the center of Washington, DC and create lat/long data variables for DC</li>
    <li>Use those lat/long to limit Foursqure Data</li>
    <li>Process Foursquare Data by neighborhood in DC</li>
    <li>Group Foursquare data into 6 categories </li>
</ol>

In [6]:

#setting DC Lat and Long-- got from Google Search
dcLat=38.5342
dcLong=-77.0211

print('center of dc')

center of dc


In [7]:
#creating the map
map_dc = folium.Map(location=[dcLat, dcLong], zoom_start=10)

#mapping neighborhoods
for lat, lng,label in zip(dc_data['Latitude'], dc_data['Longitude'], dc_data['NAME']):
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dc)
map_dc

In [8]:
CLIENT_ID = 'HJOS2S0NEUORGCKMFNVKGCB5IUX4TXNAI15POUFGTVIV1PFQ' # your Foursquare ID
CLIENT_SECRET = 'H5G2NCHJ5OKKXS5OJ3J4XT1DSS0WMDR0YKHFSJPXAVCFT205' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HJOS2S0NEUORGCKMFNVKGCB5IUX4TXNAI15POUFGTVIV1PFQ
CLIENT_SECRET:H5G2NCHJ5OKKXS5OJ3J4XT1DSS0WMDR0YKHFSJPXAVCFT205


In [9]:
dc_data.loc[0,'NAME']

'Fort Stanton'

In [10]:

dclat = dc_data.loc[0,'Latitude']
dclong = dc_data.loc[0,'Longitude']

neighborhood_name = dc_data.loc[0,'NAME']

print('Lat and Longs values of {} are {}, {}.'.format(neighborhood_name, dclat, dclong))

Lat and Longs values of Fort Stanton are 38.85565773, -76.98034771.


In [11]:
radius=500
Limit=100
print('Donzos')

Donzos


In [12]:
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    dclat,
    dclong,
    radius,
    Limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HJOS2S0NEUORGCKMFNVKGCB5IUX4TXNAI15POUFGTVIV1PFQ&client_secret=H5G2NCHJ5OKKXS5OJ3J4XT1DSS0WMDR0YKHFSJPXAVCFT205&v=20180604&ll=38.85565773,-76.98034771&radius=500&limit=100'

In [13]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9b2d43c546f3001b2ba030'},
 'response': {'headerLocation': 'Anacostia',
  'headerFullLocation': 'Anacostia, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 38.8601577345, 'lng': -76.97457984594331},
   'sw': {'lat': 38.851157725499995, 'lng': -76.9861155740567}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4cd6d6f964a52015c126e3',
       'name': 'Anacostia Community Museum',
       'location': {'address': '1901 Fort Pl SE',
        'lat': 38.85672843484337,
        'lng': -76.97689924873403,
        'labeledLatLngs': [{'label': 'display',
          'lat': 38.85672843484337,
          'lng': -76.97689924873403},
         {'label': '?', 'lat': 38.856636, 'lng': 

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Anacostia Community Museum,Museum,38.856728,-76.976899
1,Fort Stanton Recreation Center,Recreation Center,38.857118,-76.978816
2,Fort Stanton Park,Park,38.857541,-76.978266
3,Stanton Road SE & Suitland Parkway SE,Intersection,38.853278,-76.983289


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


4 venues were returned by Foursquare.


In [17]:
nearby_venues.columns

Index(['name', 'categories', 'lat', 'lng'], dtype='object')

In [18]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:

dc_venues = getNearbyVenues(names=dc_data['NAME'],
                                   latitudes=dc_data['Latitude'],
                                   longitudes=dc_data['Longitude']
                                  )

Fort Stanton
Congress Heights
Washington Highlands
Bellevue
Knox Hill/Buena Vista
Shipley
Douglass
Woodland
Garfield Heights
Near Southeast
Capitol Hill
Dupont Park
Twining
Randle Highlands
Fairlawn
Penn Branch
Barry Farm
Historic Anacostia
Columbia Heights
Logan Circle/Shaw
Cardozo/Shaw
Van Ness
Forest Hills
Georgetown Reservoir
Foxhall Village
Fort Totten
Pleasant Hill
Kenilworth
Eastland Gardens
Deanwood
Fort Dupont
Greenway
Woodland-Normanstone
Mass. Ave. Heights
Naylor Gardens
Pleasant Plains
Hillsdale
Benning Ridge
Penn Quarter
Chinatown
Stronghold
South Central
Langston
Downtown East
North Portal Estates
Colonial Village
Shepherd Park
Takoma
Lamond Riggs
Petworth
Brightwood Park
Manor Park
Brightwood
Hawthorne
Barnaby Woods
Queens Chapel
Michigan Park
North Michigan Park
Woodridge
University Heights
Brookland
Edgewood
Skyland
Bloomingdale
Lincoln Park
16th Street Heights
Fort Lincoln
Gateway
Langdon
Brentwood
Eckington
Truxton Circle
Ivy City
Trinidad
Arboretum
Carver
Mount Vern

In [20]:
print(dc_venues.shape)
dc_venues.head(20)

(2539, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fort Stanton,38.855658,-76.980348,Anacostia Community Museum,38.856728,-76.976899,Museum
1,Fort Stanton,38.855658,-76.980348,Fort Stanton Recreation Center,38.857118,-76.978816,Recreation Center
2,Fort Stanton,38.855658,-76.980348,Fort Stanton Park,38.857541,-76.978266,Park
3,Fort Stanton,38.855658,-76.980348,Stanton Road SE & Suitland Parkway SE,38.853278,-76.983289,Intersection
4,Congress Heights,38.841077,-76.997950,MLK Deli,38.843560,-76.999808,Deli / Bodega
5,Congress Heights,38.841077,-76.997950,Malcolm X and MLK,38.843144,-76.998744,Road
6,Congress Heights,38.841077,-76.997950,Alabama and MLK Ave SE,38.843196,-76.998774,Intersection
7,Congress Heights,38.841077,-76.997950,Popeyes Louisiana Kitchen,38.844926,-76.997012,Fried Chicken Joint
8,Congress Heights,38.841077,-76.997950,Rita's Italian Ice & Frozen Custard,38.843554,-76.999646,Ice Cream Shop
9,Congress Heights,38.841077,-76.997950,Alberta's Kitchen Domain,38.839236,-76.994707,American Restaurant


In [21]:
dc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
16th Street Heights,14,14,14,14,14,14
Adams Morgan,51,51,51,51,51,51
American University Park,1,1,1,1,1,1
Arboretum,16,16,16,16,16,16
Barnaby Woods,3,3,3,3,3,3
...,...,...,...,...,...,...
West End,46,46,46,46,46,46
Woodland,5,5,5,5,5,5
Woodland-Normanstone,6,6,6,6,6,6


In [22]:
print('There are {} unique categories.'.format(len(dc_venues['Venue Category'].unique())))


There are 300 unique categories.


In [23]:
#onehot encoding--need to learn more about this
ddot_onehot=pd.get_dummies(dc_venues[['Venue Category']], prefix='', prefix_sep='')

#add neigh column back to df
ddot_onehot['Neighborhood']=dc_venues['Neighborhood']
#moved neigh to the 1st
#fixed_columns=[tdot_onehot.columns[-1]] + list(ddot_onehot.columns[:-1])
#ddot_onehot=ddot_onehot[fixed_columns]


ddot_onehot.head()


,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
ddot_grouped = ddot_onehot.groupby('Neighborhood').sum().reset_index()
ddot_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,16th Street Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adams Morgan,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,American University Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arboretum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barnaby Woods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,West End,0,3,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
126,Woodland,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,Woodland-Normanstone,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128,Woodley Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


# First issue: Foursquare data is hyper-specific and not very usuable <a name="1"></a>

FourSquare has a lot of very specific labels for business, but really, I don't need to differentiate between a Mexican Restaurant and Arepa Resturant. Given I am new to python and for the sake of time, I am using a <i>brute force</i> coding method of organizing and grouping all the different types of businesses into categories more general for my purpose. 

In [26]:
for col in ddot_grouped.columns:
    print(col)

Neighborhood
Afghan Restaurant
American Restaurant
Antique Shop
Arcade
Arepa Restaurant
Art Gallery
Art Museum
Arts & Crafts Store
Arts & Entertainment
Asian Restaurant
Athletics & Sports
Automotive Shop
BBQ Joint
Bagel Shop
Bakery
Bank
Bar
Baseball Field
Basketball Court
Basketball Stadium
Bed & Breakfast
Beer Bar
Beer Garden
Beer Store
Belgian Restaurant
Big Box Store
Bike Rental / Bike Share
Bistro
Boat or Ferry
Bookstore
Border Crossing
Botanical Garden
Boutique
Brazilian Restaurant
Breakfast Spot
Brewery
Bridge
Bubble Tea Shop
Building
Burger Joint
Burrito Place
Bus Station
Bus Stop
Business Service
Cafeteria
Café
Cajun / Creole Restaurant
Camera Store
Cantonese Restaurant
Caribbean Restaurant
Check Cashing Service
Cheese Shop
Chinese Restaurant
Chocolate Shop
Clothing Store
Cocktail Bar
Coffee Shop
College Administrative Building
College Basketball Court
College Bookstore
College Library
College Quad
Comedy Club
Comfort Food Restaurant
Comic Shop
Concert Hall
Construction & Lands

In [29]:
GrFood = ddot_grouped[['Neighborhood',
                'Afghan Restaurant', 
                'American Restaurant',
                'Arepa Restaurant',
                'Asian Restaurant',
                'BBQ Joint',
                'Bagel Shop',
                'Bakery',
                'Bar',
                'Bed & Breakfast',
                'Beer Bar',
                'Beer Garden',
                'Beer Store',
                'Belgian Restaurant',
                'Bistro',
                'Brazilian Restaurant',
                'Breakfast Spot',
                'Brewery',
                'Bubble Tea Shop',
                'Burger Joint',
                'Burrito Place',
                'Cafeteria',
                'Café',
                'Cajun / Creole Restaurant',
                'Cantonese Restaurant',
                'Caribbean Restaurant',
                'Cheese Shop',
                'Chinese Restaurant',
                'Cocktail Bar',
                'Coffee Shop',
                'Comfort Food Restaurant',
                'Cuban Restaurant',
                'Cupcake Shop',
                'Deli / Bodega',
                'Dessert Shop',
                'Diner',
                'Dive Bar',
                'Donut Shop',
                'Dumpling Restaurant',
                'Eastern European Restaurant',
                'Ethiopian Restaurant',
                'Falafel Restaurant',
                'Fast Food Restaurant',
                'Filipino Restaurant',
                'Fish & Chips Shop',
                'Food',
                'Food & Drink Shop',
                'Food Service',
                'Food Truck',
                'French Restaurant',
                'Fried Chicken Joint',
                'Frozen Yogurt Shop',
                'Gastropub',
                'Gay Bar',
                'German Restaurant',
                'Gluten-free Restaurant',
                'Greek Restaurant',
                'Hookah Bar',
                'Hot Dog Joint',
                'Hotel Bar',
                'Ice Cream Shop',
                'Indian Restaurant',
                'Irish Pub',
                'Israeli Restaurant',
                'Italian Restaurant',
                'Japanese Restaurant',
                'Juice Bar',
                'Karaoke Bar',
                'Korean Restaurant',
                'Latin American Restaurant',
                'Lounge',
                'Market',
                'Mediterranean Restaurant',
                'Mexican Restaurant',
                'Middle Eastern Restaurant',
                'New American Restaurant',
                'Noodle House',
                'Peruvian Restaurant',
                'Pizza Place',
                'Poke Place',
                'Portuguese Restaurant',
                'Pub',
                'Ramen Restaurant',
                'Restaurant',
                'Roof Deck',
                'Russian Restaurant',
                'Salad Place',
                'Salvadoran Restaurant',
                'Sandwich Place',
                'Scandinavian Restaurant',
                'Seafood Restaurant',
                'Smoothie Shop',
                'Snack Place',
                'Soup Place',
                'South American Restaurant',
                'Southern / Soul Food Restaurant',
                'Spanish Restaurant',
                'Speakeasy',
                'Sports Bar',
                'Steakhouse',
                'Sushi Restaurant',
                'Taco Place',
                'Tapas Restaurant',
                'Tea Room',
                'Tex-Mex Restaurant',
                'Thai Restaurant',
                'Tiki Bar',
                'Turkish Restaurant',
                'Vegetarian / Vegan Restaurant',
                'Vietnamese Restaurant',
                'Whisky Bar',
                'Wine Bar',
                'Winery',
                'Wings Joint',
                'Xinjiang Restaurant']]
GrFood['Resturant Total']=GrFood.sum(axis=1)
GrFood.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,...,Tiki Bar,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Winery,Wings Joint,Xinjiang Restaurant,Resturant Total
0,16th Street Heights,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8
1,Adams Morgan,1,0,0,1,1,1,0,1,0,...,0,0,0,0,1,0,0,0,0,33
2,American University Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Arboretum,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,Barnaby Woods,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [41]:
GrRec = ddot_grouped[['Neighborhood',
                'Arcade',
                'Art Gallery',
                'Art Museum',
                'Athletics & Sports',
                'Baseball Field',
                'Basketball Court',
                'Basketball Stadium',
                'Bike Rental / Bike Share',
                'Botanical Garden',
                'Boutique',
                'Comedy Club',
                'Concert Hall',
                'Cycle Studio',
                'Dance Studio',
                'Dog Run',
                'Exhibit',
                'Field',
                'Fountain',
                'Garden',
                'General Entertainment',
                'Golf Course',
                'Gym',
                'Gym / Fitness Center',
                'Gym Pool',
                'Gymnastics Gym',
                'Historic Site',
                'History Museum',
                'Indie Movie Theater',
                'Jazz Club',
                'Lake',
                'Massage Studio',
                'Monument / Landmark',
                'Movie Theater',
                'Museum',
                'Music Venue',
                'Nightclub',
                'Nightlife Spot',
                'Opera House',
                'Outdoor Sculpture',
                'Park',
                'Performing Arts Venue',
                'Pilates Studio',
                'Planetarium',
                'Playground',
                'Plaza',
                'Pool',
                'Public Art',
                'Recreation Center',
                'Reservoir',
                'River',
                'Rock Club',
                'Scenic Lookout',
                'Science Museum',
                'Sculpture Garden',
                'Skating Rink',
                'Soccer Field',
                'Soccer Stadium',
                'Spa',
                'Sporting Goods Shop',
                'Sports Club',
                'State / Provincial Park',
                'Tennis Court',
                'Theater',
                'Track',
                'Trail',
                'Volleyball Court',
                'Yoga Studio',
                'Zoo Exhibit']]
GrRec['Recreation Total']=GrRec.sum(axis=1)
GrRec.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Arcade,Art Gallery,Art Museum,Athletics & Sports,Baseball Field,Basketball Court,Basketball Stadium,Bike Rental / Bike Share,Botanical Garden,...,Sports Club,State / Provincial Park,Tennis Court,Theater,Track,Trail,Volleyball Court,Yoga Studio,Zoo Exhibit,Recreation Total
0,16th Street Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,Adams Morgan,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,American University Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arboretum,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,5
4,Barnaby Woods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [32]:
GrServices = ddot_grouped[['Neighborhood',
                    'Antique Shop',
                    'Arts & Crafts Store',
                    'Automotive Shop',
                    'Bank',
                    'Big Box Store',
                    'Bookstore',
                    'Business Service',
                    'Camera Store',
                    'Clothing Store',
                    'Comic Shop',
                    'Convenience Store',
                    'Cosmetics Shop',
                    'Credit Union',
                    'Department Store',
                    'Discount Store',
                    'Drugstore',
                    'Dry Cleaner',
                    'Electronics Store',
                    'Farmers Market',
                    'Flea Market',
                    'Flower Shop',
                    'Frame Store',
                    'Furniture / Home Store',
                    'Gas Station',
                    'Gift Shop',
                    'Grocery Store',
                    'Gourmet Shop',
                    'Hardware Store',
                    'Health & Beauty Service',
                    'Home Service',
                    'Hostel',
                    'Hotel',
                    'Kitchen Supply Store',
                    'Laundromat',
                    'Leather Goods Store',
                    'Liquor Store',
                    'Mattress Store',
                    'Men\'s Store',
                    'Miscellaneous Shop',
                    'Mobile Phone Shop',
                    'Music Store',
                    'Nail Salon',
                    'Optical Shop',
                    'Organic Grocery',
                    'Other Repair Shop',
                    'Paper / Office Supplies Store',
                    'Pet Service',
                    'Photography Lab',
                    'Record Shop',
                    'Rental Car Location',
                    'Salon / Barbershop',
                    'Shipping Store',
                    'Shoe Store',
                    'Shop & Service',
                    'Shopping Mall',
                    'Shopping Plaza',
                    'Smoke Shop',
                    'Souvenir Shop',
                    'Storage Facility',
                    'Supermarket',
                    'Supplement Shop',
                    'Tailor Shop',
                    'Thrift / Vintage Store',
                    'Toy / Game Store',
                    'Video Store',
                    'Warehouse Store',
                    'Wine Shop',
                    'Women\'s Store']]
GrServices['Services Total']=GrServices.sum(axis=1)
GrServices.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Antique Shop,Arts & Crafts Store,Automotive Shop,Bank,Big Box Store,Bookstore,Business Service,Camera Store,Clothing Store,...,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store,Services Total
0,16th Street Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Adams Morgan,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,9
2,American University Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arboretum,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,Barnaby Woods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
Grcommute = ddot_grouped[['Neighborhood',
                    'Bus Station',
                    'Bus Stop',
                    'Light Rail Station',
                    'Metro Station',
                    'Train Station',]]
Grcommute['Commuting Total']=Grcommute.sum(axis=1)
Grcommute.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Neighborhood,Bus Station,Bus Stop,Light Rail Station,Metro Station,Train Station,Commuting Total
0,16th Street Heights,0,1,0,0,0,1
1,Adams Morgan,0,0,0,0,0,0
2,American University Park,0,0,0,0,0,0
3,Arboretum,0,0,0,0,0,0
4,Barnaby Woods,0,0,0,0,0,0


In [36]:
GrMedical = ddot_grouped[['Neighborhood',
                    'Hospital',
                    'Pharmacy',]]
GrMedical['Medical Total']=GrMedical.sum(axis=1)
GrMedical.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Neighborhood,Hospital,Pharmacy,Medical Total
0,16th Street Heights,0,0,0
1,Adams Morgan,0,0,0
2,American University Park,0,0,0
3,Arboretum,0,0,0
4,Barnaby Woods,0,0,0


In [37]:
GrEdu = ddot_grouped[['Neighborhood',
                'College Administrative Building',
                'College Bookstore',
                'College Library',
                'College Quad',
                'General College & University']]
GrEdu['Education Total']=GrEdu.sum(axis=1)
GrEdu.head()

/Users/jeremymcdonald/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Neighborhood,College Administrative Building,College Bookstore,College Library,College Quad,General College & University,Education Total
0,16th Street Heights,0,0,0,0,0,0
1,Adams Morgan,0,0,0,0,0,0
2,American University Park,0,0,0,0,0,0
3,Arboretum,0,0,0,0,0,0
4,Barnaby Woods,0,0,0,0,0,0


# Solution to problem 1 
I admit the above is probably not the most elgant or advance way to total a group of columns and the code in the next section is equally problematic. For me, doing the long and messy way was instructive in how dataframes coloumns and rows works. So rather then going back and cleaning this up and creating a more advance process-- I am leaving it as is.

# Next steps

To make the data usable, I am going to total the <i>Brute Force</i> categories I created above. These totals will ultimaly allow me to rank order my 13 unique locations

For both learning and documentation reasons, I am doing each category one at a time. So in each set of code, I am adding the "total" for the Foursquare group to a dataframe, renaming the dataframe, and moving to the next category. 

In [42]:
dfFood=GrFood[['Neighborhood','Resturant Total']]
dfRec=GrRec[['Neighborhood','Recreation Total']]
dfServices=GrServices[['Neighborhood','Services Total']]
dfCommute=Grcommute[['Neighborhood','Commuting Total']]
dfEdu=GrEdu[['Neighborhood','Education Total']]
dfMed=GrMedical[['Neighborhood','Medical Total']]

In [43]:
df_index_col=pd.merge(dfFood, dfRec, on='Neighborhood')

df_index_col

,Neighborhood,Resturant Total,Recreation Total
0,16th Street Heights,8,4
1,Adams Morgan,33,8
2,American University Park,1,0
3,Arboretum,6,5
4,Barnaby Woods,1,2
...,...,...,...
125,West End,23,10
126,Woodland,1,4
127,Woodland-Normanstone,1,4
128,Woodley Park,7,8


In [44]:
df_index_1=pd.merge(df_index_col, dfServices, on='Neighborhood')

df_index_1

,Neighborhood,Resturant Total,Recreation Total,Services Total
0,16th Street Heights,8,4,1
1,Adams Morgan,33,8,9
2,American University Park,1,0,0
3,Arboretum,6,5,4
4,Barnaby Woods,1,2,0
...,...,...,...,...
125,West End,23,10,11
126,Woodland,1,4,0
127,Woodland-Normanstone,1,4,0
128,Woodley Park,7,8,5


In [45]:
df_index_2=pd.merge(df_index_1, dfCommute, on='Neighborhood')

df_index_2

,Neighborhood,Resturant Total,Recreation Total,Services Total,Commuting Total
0,16th Street Heights,8,4,1,1
1,Adams Morgan,33,8,9,0
2,American University Park,1,0,0,0
3,Arboretum,6,5,4,0
4,Barnaby Woods,1,2,0,0
...,...,...,...,...,...
125,West End,23,10,11,0
126,Woodland,1,4,0,0
127,Woodland-Normanstone,1,4,0,0
128,Woodley Park,7,8,5,1


In [46]:
df_index_3=pd.merge(df_index_2, dfEdu, on='Neighborhood')

df_index_3

,Neighborhood,Resturant Total,Recreation Total,Services Total,Commuting Total,Education Total
0,16th Street Heights,8,4,1,1,0
1,Adams Morgan,33,8,9,0,0
2,American University Park,1,0,0,0,0
3,Arboretum,6,5,4,0,0
4,Barnaby Woods,1,2,0,0,0
...,...,...,...,...,...,...
125,West End,23,10,11,0,0
126,Woodland,1,4,0,0,0
127,Woodland-Normanstone,1,4,0,0,0
128,Woodley Park,7,8,5,1,0


In [47]:
df_index_4=pd.merge(df_index_3, dfMed, on='Neighborhood')

df_index_4

,Neighborhood,Resturant Total,Recreation Total,Services Total,Commuting Total,Education Total,Medical Total
0,16th Street Heights,8,4,1,1,0,0
1,Adams Morgan,33,8,9,0,0,0
2,American University Park,1,0,0,0,0,0
3,Arboretum,6,5,4,0,0,0
4,Barnaby Woods,1,2,0,0,0,0
...,...,...,...,...,...,...,...
125,West End,23,10,11,0,0,0
126,Woodland,1,4,0,0,0,0
127,Woodland-Normanstone,1,4,0,0,0,0
128,Woodley Park,7,8,5,1,0,1


# Issue 2: Matching my location File with Foursquare Neighborhood File <a name="2"></a>

My location file is attached to clusters, not Neighborhoods because the nature of DC's geography. DC is comprised of 4 sections (Northwest, Northeast, Southwest, Southeast) and 8 Wards(Ward1-Ward8). In addition, DC has many unique and changing Neighborhoods. The problem -- wards are not confined to sections and neighborhoods are not confined to wards. So a Neighborhood could be in multiple wards and sections. DC government provides many KML files to help people with the interesting geographic organization in the city, the problem, the KML file for Neighborhoods was not polygons, rather points. So I had no way of testing which neighborhood my location was in, BUT I could use the KML file for Cluster. Cluster is another way DC organizes its geography. I wanted to us python to test if my point is inside of a cluster, but ran into some issues. To save time, I used QGIS and was able to create two files dfNeihborhood and dfLocation, which mapped DC's neighborhoods to clusters and my locations to Clusters. Now I have a common point to connect the two.


Fun fact- the sections in DC are important for an address. As in some cases there are four different places with the same address, but different sections. Even more fun facts-- the 4 sections meet in the geographic center of the US Capitol. So the closer each address is to the US Capitol, the closer it is to the duplicate address. Always check NW, NE, SW, SE to make sure you know!

In [48]:
#load locations to analyze and neighborhood connected to cluster
df_N= 'dfNeighborhood.csv'
Neighborhood_data=pd.read_csv(df_N)


dc_L='dfLocation.csv'
location_data=pd.read_csv(dc_L)
print ('locked and loaded')

locked and loaded


In [49]:
#create a new dataframe to join the Foursquare data too. 
dfMaster=df_index_4[['Neighborhood']]

df_index_5=pd.merge(Neighborhood_data,dfMaster, on='Neighborhood')

df_index_5

,OBJECTID,GIS_ID,Neighborhood,Cluster Name
0,1,nhood_050,Fort Stanton,Cluster 37
1,2,nhood_031,Congress Heights,Cluster 39
2,3,nhood_123,Washington Highlands,Cluster 39
3,4,nhood_008,Bellevue,Cluster 39
4,5,nhood_073,Knox Hill/Buena Vista,Cluster 36
...,...,...,...,...
125,128,nhood_046,Fort Davis Park,Cluster 34
126,129,nhood_042,Fairfax Village,Cluster 35
127,130,nhood_064,Hillcrest,Cluster 35
128,131,nhood_033,Crestwood,Cluster 18


In [50]:
#add select locations to dfMaster
df_index_6=pd.merge(df_index_5, location_data, on='Cluster Name')
#clean up the end of the dataframe
df_index_6.drop(['Street', 'City','State','Zip', 'Latitude', 'Longitude'], axis=1, inplace=True)
df_index_6

,OBJECTID,GIS_ID,Neighborhood,Cluster Name,Location
0,2,nhood_031,Congress Heights,Cluster 39,Theta
1,3,nhood_123,Washington Highlands,Cluster 39,Theta
2,4,nhood_008,Bellevue,Cluster 39,Theta
3,11,nhood_021,Capitol Hill,Cluster 26,Gamma
4,65,nhood_135,Lincoln Park,Cluster 26,Gamma
5,116,nhood_079,Hill East,Cluster 26,Gamma
6,18,nhood_066,Historic Anacostia,Cluster 28,Alpha
7,20,nhood_080,Logan Circle/Shaw,Cluster 7,Epsilon
8,39,nhood_130,Penn Quarter,Cluster 8,Iota
9,40,nhood_131,Chinatown,Cluster 8,Iota


In [51]:
#File showing neighborhood with counts and location. 
df_index_7=pd.merge(df_index_6, df_index_4, on='Neighborhood')
#More Cleanup of dataframe
df_index_7.drop(['OBJECTID', 'GIS_ID'], axis=1, inplace=True)
df_index_7

,Neighborhood,Cluster Name,Location,Resturant Total,Recreation Total,Services Total,Commuting Total,Education Total,Medical Total
0,Congress Heights,Cluster 39,Theta,5,1,4,0,0,0
1,Washington Highlands,Cluster 39,Theta,3,1,3,0,0,0
2,Bellevue,Cluster 39,Theta,2,3,1,0,0,0
3,Capitol Hill,Cluster 26,Gamma,60,20,18,0,0,0
4,Lincoln Park,Cluster 26,Gamma,1,2,2,1,0,0
5,Hill East,Cluster 26,Gamma,1,3,0,0,0,0
6,Historic Anacostia,Cluster 28,Alpha,7,3,2,0,0,0
7,Logan Circle/Shaw,Cluster 7,Epsilon,26,5,2,1,0,0
8,Penn Quarter,Cluster 8,Iota,60,32,7,0,0,0
9,Chinatown,Cluster 8,Iota,64,21,11,0,1,0


In [52]:
#combine locations into single row
df_Final=df_index_7.groupby('Location').sum()
#Dropping Commuting total because the number is off-- instead I am going to add WMATA data instead
df_Final.drop(['Commuting Total'], axis=1, inplace=True)
df_Final

,Resturant Total,Recreation Total,Services Total,Education Total,Medical Total
Location,,,,,
Alpha,7,3,2,0,0
Beta,28,17,12,0,1
Delta,26,15,18,0,1
Epsilon,26,5,2,0,0
Eta,32,8,10,0,1
Gamma,62,25,20,0,0
Iota,272,89,68,3,4
Kappa,38,15,31,0,1
Mu,25,9,16,0,2


# Issue 3: WMATA Data <a name="3"></a>

The data I categorized as "commuter" was not correct. A bunch of locations came up with little to no commuter options. Within Washington, DC there is an extensive mass transit system of buses and trains. The bus system is everywhere and really not useful in a ranking system-- each location has ample access to buses. 
Trains on anotehr hand are slightly different. Large sections of Washington, DC do not have access to a "metro" (locals general refer to metro trains as <i>metro</i> and metro buses as <i>buses</i>).
As noted above, Foursquare really did not capture this data. Therefore, I pull in WMATA data using their free API to identify where each train station is and the accessibility of my 13 unique locations to those stations 

In [53]:
#Just looking at the data, I can see that the Commuting data points are off. 
#There are a bunch of metro stations and even more buslines

#We are going to get a list of Metro stations from WMATA's free JSON connection

import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'api_key': '38dcc021678b4ee99f3322264890a860',
}

params = urllib.parse.urlencode({
    # Request parameters
    'LineCode': '',
})

try:
    conn = http.client.HTTPSConnection('api.wmata.com')
    conn.request("GET", "/Rail.svc/json/jStations?%s" % params, "{body}", headers)
    response = conn.getresponse()
    json_data = response.read()
    print(json_data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


b'{"Stations":[{"Code":"A01","Name":"Metro Center","StationTogether1":"C01","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.898303,"Lon":-77.028099,"Address":{"Street":"607 13th St. NW","City":"Washington","State":"DC","Zip":"20005"}},{"Code":"A02","Name":"Farragut North","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.903192,"Lon":-77.039766,"Address":{"Street":"1001 Connecticut Avenue NW","City":"Washington","State":"DC","Zip":"20036"}},{"Code":"A03","Name":"Dupont Circle","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.909499,"Lon":-77.04362,"Address":{"Street":"1525 20th St. NW","City":"Washington","State":"DC","Zip":"20036"}},{"Code":"A04","Name":"Woodley Park-Zoo/Adams Morgan","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null

In [54]:
json_data

b'{"Stations":[{"Code":"A01","Name":"Metro Center","StationTogether1":"C01","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.898303,"Lon":-77.028099,"Address":{"Street":"607 13th St. NW","City":"Washington","State":"DC","Zip":"20005"}},{"Code":"A02","Name":"Farragut North","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.903192,"Lon":-77.039766,"Address":{"Street":"1001 Connecticut Avenue NW","City":"Washington","State":"DC","Zip":"20036"}},{"Code":"A03","Name":"Dupont Circle","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null,"Lat":38.909499,"Lon":-77.04362,"Address":{"Street":"1525 20th St. NW","City":"Washington","State":"DC","Zip":"20036"}},{"Code":"A04","Name":"Woodley Park-Zoo/Adams Morgan","StationTogether1":"","StationTogether2":"","LineCode1":"RD","LineCode2":null,"LineCode3":null,"LineCode4":null

In [55]:
#I need to serealize the json data to use it
wmata_data = json.loads(json_data)
print(wmata_data)

{'Stations': [{'Code': 'A01', 'Name': 'Metro Center', 'StationTogether1': 'C01', 'StationTogether2': '', 'LineCode1': 'RD', 'LineCode2': None, 'LineCode3': None, 'LineCode4': None, 'Lat': 38.898303, 'Lon': -77.028099, 'Address': {'Street': '607 13th St. NW', 'City': 'Washington', 'State': 'DC', 'Zip': '20005'}}, {'Code': 'A02', 'Name': 'Farragut North', 'StationTogether1': '', 'StationTogether2': '', 'LineCode1': 'RD', 'LineCode2': None, 'LineCode3': None, 'LineCode4': None, 'Lat': 38.903192, 'Lon': -77.039766, 'Address': {'Street': '1001 Connecticut Avenue NW', 'City': 'Washington', 'State': 'DC', 'Zip': '20036'}}, {'Code': 'A03', 'Name': 'Dupont Circle', 'StationTogether1': '', 'StationTogether2': '', 'LineCode1': 'RD', 'LineCode2': None, 'LineCode3': None, 'LineCode4': None, 'Lat': 38.909499, 'Lon': -77.04362, 'Address': {'Street': '1525 20th St. NW', 'City': 'Washington', 'State': 'DC', 'Zip': '20036'}}, {'Code': 'A04', 'Name': 'Woodley Park-Zoo/Adams Morgan', 'StationTogether1': '

In [56]:
#Now the json data is in a usable format
print(json.dumps(wmata_data, indent = 4, sort_keys=True))

{
    "Stations": [
        {
            "Address": {
                "City": "Washington",
                "State": "DC",
                "Street": "607 13th St. NW",
                "Zip": "20005"
            },
            "Code": "A01",
            "Lat": 38.898303,
            "LineCode1": "RD",
            "LineCode2": null,
            "LineCode3": null,
            "LineCode4": null,
            "Lon": -77.028099,
            "Name": "Metro Center",
            "StationTogether1": "C01",
            "StationTogether2": ""
        },
        {
            "Address": {
                "City": "Washington",
                "State": "DC",
                "Street": "1001 Connecticut Avenue NW",
                "Zip": "20036"
            },
            "Code": "A02",
            "Lat": 38.903192,
            "LineCode1": "RD",
            "LineCode2": null,
            "LineCode3": null,
            "LineCode4": null,
            "Lon": -77.039766,
            "Name": "Farragut Nort

In [57]:
#cleaning JSON Data
#ignore my naming convetions, I was having an issue, so went back to the lab and found it easier to stick with orig

neighborhoods_data = wmata_data['Stations']

In [58]:
neighborhoods_data

[{'Code': 'A01',
  'Name': 'Metro Center',
  'StationTogether1': 'C01',
  'StationTogether2': '',
  'LineCode1': 'RD',
  'LineCode2': None,
  'LineCode3': None,
  'LineCode4': None,
  'Lat': 38.898303,
  'Lon': -77.028099,
  'Address': {'Street': '607 13th St. NW',
   'City': 'Washington',
   'State': 'DC',
   'Zip': '20005'}},
 {'Code': 'A02',
  'Name': 'Farragut North',
  'StationTogether1': '',
  'StationTogether2': '',
  'LineCode1': 'RD',
  'LineCode2': None,
  'LineCode3': None,
  'LineCode4': None,
  'Lat': 38.903192,
  'Lon': -77.039766,
  'Address': {'Street': '1001 Connecticut Avenue NW',
   'City': 'Washington',
   'State': 'DC',
   'Zip': '20036'}},
 {'Code': 'A03',
  'Name': 'Dupont Circle',
  'StationTogether1': '',
  'StationTogether2': '',
  'LineCode1': 'RD',
  'LineCode2': None,
  'LineCode3': None,
  'LineCode4': None,
  'Lat': 38.909499,
  'Lon': -77.04362,
  'Address': {'Street': '1525 20th St. NW',
   'City': 'Washington',
   'State': 'DC',
   'Zip': '20036'}},
 {

In [59]:
# define the dataframe column
column_names = ['Name', 'Lat', 'Lon', 'Code'] 

# instantiate the dataframe
stations = pd.DataFrame(columns=column_names)

stations

,Name,Lat,Lon,Code


In [61]:
for data in neighborhoods_data:
    borough = data['Name']
    neighborhood_name = data['Code']
    neighborhood_lat = data['Lat']
    neighborhood_lon = data['Lon']
        
    
    stations = stations.append({'Name': borough,
                                          'Code': neighborhood_name,
                                          'Lat': neighborhood_lat,
                                          'Lon': neighborhood_lon}, ignore_index=True)

In [62]:
stations.head()

,Name,Lat,Lon,Code
0,Metro Center,38.898303,-77.028099,A01
1,Farragut North,38.903192,-77.039766,A02
2,Dupont Circle,38.909499,-77.043620,A03
3,Woodley Park-Zoo/Adams Morgan,38.924999,-77.052648,A04
4,Cleveland Park,38.934703,-77.058226,A05


# Issue 4 Python GIS <a name="4"></a>
My GIS python code just would not work. I ran into problem after problem, so I utlized KML files from the DC Government and a program called QGIS to create csv files to help map my location data and import that into Dataframes.

In [63]:
#My KLM and Python skills need work, So I'm exporting this out to QGIS to attach cluster to metro station
export_Metro = stations.to_csv(r'Root_dir', index = None, header = True)

print('Exported')

Exported


In [64]:
#import the new data file back in
stations_data=pd.read_csv('WMATA_Cluster.csv')

stations_data.head()

,Cluster Name,Sum of Metro Stops
0,Cluster 11,2
1,Cluster 12,1
2,Cluster 15,2
3,Cluster 17,1
4,Cluster 18,1


In [65]:
#cleaning the file so I just have metro stops
stations_final=stations_data.groupby('Cluster Name').sum()

stations_final.head()

,Sum of Metro Stops
Cluster Name,
Cluster 11,2
Cluster 12,1
Cluster 15,2
Cluster 17,1
Cluster 18,1


In [66]:
#I am creating a table that has location and cluster # so I can add final to it
loc=location_data


loc


,Location,Street,City,State,Zip,Latitude,Longitude,Cluster Name
0,Alpha,1604 Morris Rd SE,Washington,DC,20020,38.858122,-76.982118,Cluster 28
1,Beta,320 21st St NE,Washington,DC,20002,38.894234,-76.975460,Cluster 25
2,Gamma,"1503 East Capitol Street, SE",Washington,DC,20003,38.889428,-76.983260,Cluster 26
3,Delta,"1217 West Virginia Ave, NE",Washington,DC,20002,38.905281,-76.991165,Cluster 23
4,Epsilon,"711 N Street, NW",Washington,DC,20001,38.907529,-77.022662,Cluster 7
5,Zeta,"2019 Rhode Island Ave, NE",Washington,DC,20018,38.928684,-76.974973,Cluster 22
6,Eta,"510 Webster St., NW",Washington,DC,20011,38.943775,-77.020883,Cluster 18
7,Theta,"220 High View Place, SE",Washington,DC,20032,38.841594,-77.004447,Cluster 39
8,Iota,"1135 New Jersey Ave, NW",Washington,DC,20001,38.904889,-77.014154,Cluster 8
9,Kappa,"4404 Wisconsin Ave, NW",Washington,DC,20016,38.946962,-77.079989,Cluster 11


In [67]:
#add Metro stop data
df_Final2=pd.merge(loc, df_Final, on='Location')
#Add metro
df_Final3=pd.merge(df_Final2, stations_final, on='Cluster Name')
#clean up the end of the dataframe

df_Final3


,Location,Street,City,State,Zip,Latitude,Longitude,Cluster Name,Resturant Total,Recreation Total,Services Total,Education Total,Medical Total,Sum of Metro Stops
0,Beta,320 21st St NE,Washington,DC,20002,38.894234,-76.975460,Cluster 25,28,17,12,0,1,2
1,Gamma,"1503 East Capitol Street, SE",Washington,DC,20003,38.889428,-76.983260,Cluster 26,62,25,20,0,0,4
2,Epsilon,"711 N Street, NW",Washington,DC,20001,38.907529,-77.022662,Cluster 7,26,5,2,0,0,1
3,Zeta,"2019 Rhode Island Ave, NE",Washington,DC,20018,38.928684,-76.974973,Cluster 22,6,7,12,0,0,1
4,Eta,"510 Webster St., NW",Washington,DC,20011,38.943775,-77.020883,Cluster 18,32,8,10,0,1,1
5,Iota,"1135 New Jersey Ave, NW",Washington,DC,20001,38.904889,-77.014154,Cluster 8,272,89,68,3,4,8
6,Kappa,"4404 Wisconsin Ave, NW",Washington,DC,20016,38.946962,-77.079989,Cluster 11,38,15,31,0,1,2
7,Mu,"6008 Georgia Ave, NW",Washington,DC,20011,38.963195,-77.028373,Cluster 17,25,9,16,0,2,1


# Results <a name="results"></a>

I have the columns that I will use to identify the best locations, to do that I am going to rank each column, and order by importance.

In [68]:
# Create a column Rating_Rank which contains 
# the rank of each movie based on rating 
df_Final3['Rest_Rank'] = df_Final3['Resturant Total'].rank(ascending = 1)
df_Final3['Rec_Rank'] = df_Final3['Recreation Total'].rank(ascending = 1)
df_Final3['Serv_Rank'] = df_Final3['Services Total'].rank(ascending =1)
df_Final3['Ed_Rank'] = df_Final3['Education Total'].rank(ascending=1)
df_Final3['Med_Rank'] = df_Final3['Medical Total'].rank(ascending=1)
df_Final3['Metro_Rank']= df_Final3['Sum of Metro Stops'].rank(ascending=1)


print(df_Final3) 


  Location                        Street        City State    Zip   Latitude  \
0     Beta                320 21st St NE  Washington    DC  20002  38.894234   
1    Gamma  1503 East Capitol Street, SE  Washington    DC  20003  38.889428   
2  Epsilon              711 N Street, NW  Washington    DC  20001  38.907529   
3     Zeta     2019 Rhode Island Ave, NE  Washington    DC  20018  38.928684   
4      Eta           510 Webster St., NW  Washington    DC  20011  38.943775   
5     Iota       1135 New Jersey Ave, NW  Washington    DC  20001  38.904889   
6    Kappa        4404 Wisconsin Ave, NW  Washington    DC  20016  38.946962   
7       Mu          6008 Georgia Ave, NW  Washington    DC  20011  38.963195   

   Longitude Cluster Name  Resturant Total  Recreation Total  Services Total  \
0 -76.975460   Cluster 25               28                17              12   
1 -76.983260   Cluster 26               62                25              20   
2 -77.022662    Cluster 7              

Now, I am goint to weight Metro, Resturant, and Education so that they are "worth more"
Metro= X10
Resturant = X5
Education = X2

In [71]:
df_Final3['Metro']=df_Final3['Metro_Rank']*10
df_Final3['Resturant']=df_Final3['Rest_Rank']*5
df_Final3['Ed']=df_Final3['Ed_Rank']*2

df_Final3.head()

,Location,Street,City,State,Zip,Latitude,Longitude,Cluster Name,Resturant Total,Recreation Total,...,Rest_Rank,Rec_Rank,Serv_Rank,Ed_Rank,Med_Rank,Metro_Rank,Metro,Resturant,Ed,FINAL
0,Beta,320 21st St NE,Washington,DC,20002,38.894234,-76.975460,Cluster 25,28,17,...,4.0,6.0,3.5,4.0,5.0,5.5,55.0,20.0,8.0,97.5
1,Gamma,"1503 East Capitol Street, SE",Washington,DC,20003,38.889428,-76.983260,Cluster 26,62,25,...,7.0,7.0,6.0,4.0,2.0,7.0,70.0,35.0,8.0,128.0
2,Epsilon,"711 N Street, NW",Washington,DC,20001,38.907529,-77.022662,Cluster 7,26,5,...,3.0,1.0,1.0,4.0,2.0,2.5,25.0,15.0,8.0,52.0
3,Zeta,"2019 Rhode Island Ave, NE",Washington,DC,20018,38.928684,-76.974973,Cluster 22,6,7,...,1.0,2.0,3.5,4.0,2.0,2.5,25.0,5.0,8.0,45.5
4,Eta,"510 Webster St., NW",Washington,DC,20011,38.943775,-77.020883,Cluster 18,32,8,...,5.0,3.0,2.0,4.0,5.0,2.5,25.0,25.0,8.0,68.0


Now I am going to create a simple column that adds the non-weighted columns to the three weighted columns, creating what is my final ranking for each location. 

In [72]:
df_Final3['FINAL']= df_Final3['Rec_Rank']+df_Final3['Serv_Rank']+df_Final3['Med_Rank']+df_Final3['Metro']+df_Final3['Resturant']+df_Final3['Ed']

df_Final3.head()

,Location,Street,City,State,Zip,Latitude,Longitude,Cluster Name,Resturant Total,Recreation Total,...,Rest_Rank,Rec_Rank,Serv_Rank,Ed_Rank,Med_Rank,Metro_Rank,Metro,Resturant,Ed,FINAL
0,Beta,320 21st St NE,Washington,DC,20002,38.894234,-76.975460,Cluster 25,28,17,...,4.0,6.0,3.5,4.0,5.0,5.5,55.0,20.0,8.0,97.5
1,Gamma,"1503 East Capitol Street, SE",Washington,DC,20003,38.889428,-76.983260,Cluster 26,62,25,...,7.0,7.0,6.0,4.0,2.0,7.0,70.0,35.0,8.0,128.0
2,Epsilon,"711 N Street, NW",Washington,DC,20001,38.907529,-77.022662,Cluster 7,26,5,...,3.0,1.0,1.0,4.0,2.0,2.5,25.0,15.0,8.0,52.0
3,Zeta,"2019 Rhode Island Ave, NE",Washington,DC,20018,38.928684,-76.974973,Cluster 22,6,7,...,1.0,2.0,3.5,4.0,2.0,2.5,25.0,5.0,8.0,45.5
4,Eta,"510 Webster St., NW",Washington,DC,20011,38.943775,-77.020883,Cluster 18,32,8,...,5.0,3.0,2.0,4.0,5.0,2.5,25.0,25.0,8.0,68.0


In [73]:
df_Final3=df_Final3.set_index('FINAL')
df_Final3=df_Final3.sort_index()

print(df_Final3)

      Location                        Street        City State    Zip  \
FINAL                                                                   
45.5      Zeta     2019 Rhode Island Ave, NE  Washington    DC  20018   
52.0   Epsilon              711 N Street, NW  Washington    DC  20001   
59.0        Mu          6008 Georgia Ave, NW  Washington    DC  20011   
68.0       Eta           510 Webster St., NW  Washington    DC  20011   
97.5      Beta                320 21st St NE  Washington    DC  20002   
110.0    Kappa        4404 Wisconsin Ave, NW  Washington    DC  20016   
128.0    Gamma  1503 East Capitol Street, SE  Washington    DC  20003   
160.0     Iota       1135 New Jersey Ave, NW  Washington    DC  20001   

        Latitude  Longitude Cluster Name  Resturant Total  Recreation Total  \
FINAL                                                                         
45.5   38.928684 -76.974973   Cluster 22                6                 7   
52.0   38.907529 -77.022662    C

# FINAL OUTCOME <a name="conclusion"></a>

After pulling data from:
<ol type="1">
    <li>Foursquare</li>
    <li><i>Brute Force</i> categorizing Foursquare Data</li>
    <li>Pulling WMATA Data</li>
    <li>Using QGIS to map and properly identify which neighborhood and cluster the 13 unique locations, Foursquare, and WMATA Data are in</li>
    <li>Creating a dataframe with weighted totals for differenty ammenities for each location</li>
    <li>Rank ordering</li>
</ol>
The winner is:
With a total ranking of 160, location <big>IOTA</big> is the winner. 